In [16]:
# imports and settings
from __future__ import division
import pandas as pd
import numpy as np
import os
from os.path import join as opj
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>")) # cell wdith at 80% of window

pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 1000

# Ensure correct wokring directory
if os.getcwd().split('/')[-1].startswith('IMCN-SST') == True:
    print('In the correct working directory: ' + str(os.getcwd()))
else:
    os.chdir("..")
    print('Changed to correct working directory: ' + str(os.getcwd()))

In the correct working directory: /Users/scotti/surfdrive/Shared/NTNU_fMRI_tasks/tasks_for_at_NTNU/IMCN-SST-visual


# Settings

Define the main parameters for the design file of the experiment

Important:
- Number of trials in each block MUST be even


Designs must have the following columns (in any order):

trial_ID, direction, stop_trial, jitter, block, subject, stimulus_dur, feedback_dur, null_trial, ITI, n_trs

In [35]:
def get_settings(tr=2, n_blocks=3, n_sessions=1, n_trials_per_block=128, p_null_trials=0.1, 
                 p_stop_trial=0.25, volumes_per_trial=4, verbose=True):
    
    jitters = [0.5, 1.0, 1.5, 2.0]                                         # Define the jitter times
    
    if p_null_trials == 0:
        n_null_trials = 0
    else:
        n_null_trials = round(p_null_trials*n_trials_per_block)                # number of null trials
#         if (n_null_trials % 2) != 0:
#             n_null_trials = n_null_trials-1
    n_trials_per_block_not_null = n_trials_per_block-n_null_trials         # Number of trials that are not null trials
    total_trials_all_blocks = n_trials_per_block*n_blocks                  # total trials
    n_stop_trials = int(np.ceil(n_trials_per_block_not_null)*p_stop_trial) # number of stop trials
    n_go_trials = n_trials_per_block_not_null - n_stop_trials              # number of go trials
    perc = (n_stop_trials / n_trials_per_block_not_null)*100               # percentage of trials that are stop trials
    trial_duration = round(volumes_per_trial*tr,3)                         # duration of one trial
    total_block_duration = trial_duration*n_trials_per_block               # duration of one block (s)
    total_duration = total_block_duration*n_blocks                         # total duration (s)
    total_duration_min = total_duration/60                                 # total duration (min)
    total_duration_one_block = total_duration_min/n_blocks                 # duration of one block (min)
    total_trials = n_trials_per_block*n_blocks                             # total number of trials
    total_volumes = 1 + n_trials_per_block*volumes_per_trial*n_blocks      # total number of volumes
    total_stop_trials = n_stop_trials*n_blocks                             # total number of stop trials
    
    if verbose:
        print(f'''Settings:\n\n\
        Sessions: {n_sessions}\n\
        Blocks per session: {n_blocks}\n\
        Trials per block: {n_trials_per_block}\n\
        Total trials entire session: {total_trials_all_blocks}\n\
        Stop trials per block: {n_stop_trials} ({perc:.2f} percent)\n\
        Number of null trials per block: {p_null_trials}*{n_trials_per_block} = {n_null_trials}\n\
        Assuming a TR of {tr} seconds\n\
        Jitter options: {jitters} seconds\n\
        Total duration: {round(total_duration,3)} seconds\n\
        Total duration: {round(total_duration_min,3)} minutes\n\
        Total volumes: {total_volumes} volumes\n\
        Total stop trials: {n_stop_trials}*{n_blocks} = {total_stop_trials}\n\
        Total duration of one block: {round(trial_duration,3)}*{n_trials_per_block} = {round(total_duration,3)} seconds = {round(total_duration_one_block,3)} min\n\
        Total number of volumes necessary for 1 block: 1+{n_trials_per_block}*{volumes_per_trial} = {total_volumes} + warm-up pulses''')
        
    return({'jitter': jitters,
            'n_stop_trials': n_stop_trials,
            'n_go_trials': n_go_trials,
            'n_trials_per_block': n_trials_per_block,
            'n_trials_per_block_not_null': n_trials_per_block_not_null,
            'n_null_trials': n_null_trials,
            'n_blocks': n_blocks,
            'session_tr': tr,
            'trial_length': trial_duration,
            'n_trs': volumes_per_trial})

Check the settings with some usable parameters

In [29]:
settings=get_settings(tr=1.4, n_blocks=3, n_sessions=1, n_trials_per_block=128, p_null_trials=0.1, 
                     p_stop_trial=0.25, volumes_per_trial=6, verbose=True)
settings

Settings:

        Sessions: 1
        Blocks per session: 3
        Trials per block: 128
        Total trials entire session: 384
        Stop trials per block: 29 (25.00 percent)
        Number of null trials per block: 0.1*128 = 12
        Assuming a TR of 1.4 seconds
        Jitter options: [0.5, 1.0, 1.5, 2.0] seconds
        Total duration: 3225.6 seconds
        Total duration: 53.76 minutes
        Total volumes: 2305 volumes
        Total stop trials: 29*3 = 87
        Total duration of one block: 8.4*128 = 3225.6 seconds = 17.92 min
        Total number of volumes necessary for 1 block: 1+128*6 = 2305 + warm-up pulses


{'jitter': [0.5, 1.0, 1.5, 2.0],
 'n_stop_trials': 29,
 'n_go_trials': 87,
 'n_trials_per_block': 128,
 'n_trials_per_block_not_null': 116,
 'n_null_trials': 12,
 'n_blocks': 3,
 'session_tr': 1.4,
 'trial_length': 8.4,
 'n_trs': 6}

Define function to generate the designs

In [19]:
# generate design function
def generate_design(n_trials_per_block, jitters, n_stop_trials, n_null_trials):
    
    if n_null_trials is not None:
        n_not_null_trials = n_trials_per_block - n_null_trials
        n_go_trials = n_not_null_trials - n_stop_trials
    else:
        n_not_null_trials = n_trials_per_block
        n_go_trials = n_not_null_trials-n_stop_trials
            
    design = pd.DataFrame({'direction': np.tile([0, 1], int(np.ceil(n_not_null_trials/2))), # arrow direction
                           'stop_trial': 0})#,                  # stop trial
    design.loc[:(n_stop_trials-1), 'stop_trial'] = 1            # add correct number of stop trials
    
    jitts_to_add = np.random.choice(jitters, size=n_not_null_trials, replace=True)
    design['jitter'] = jitts_to_add
        
    design = design.sample(frac=1).reset_index(drop=True) # Mix up design each while loop iteration

    return(design)

Define other important functions for design creation

In [20]:
# Add in pseudorandomizer class
class Pseudorandomizer(object):
    
    #def __init__(self, data, max_identical_iters={'cue': 4, 'correct_answer': 4}):  
    def __init__(self, data, max_identical_iters={'direction': 3, 'stop_trial': 2}):  
        self.data = data
        self.max_identical_iters = {x: y+1 for x, y in list(max_identical_iters.items())}
                                    # add 1: if 4 rows is allowed, only give an error after 5 identical rows
    
    def check_trial_rows(self, data, row_n): 
        """
        Returns True if any of the conditions for pseudorandomization are violated for the given rows, 
        False if they are fine.
        """
        
        # First, check for the maximum iterations
        for column, max_iter in list(self.max_identical_iters.items()):
            if row_n - max_iter < 0:
                continue

            # Select rows [max_iter-1 - row_n] we're going to check. Never select any row with index < 0
            row_selection = [x for x in np.arange(row_n, row_n-max_iter, -1)]

            # Next, we check if the selected rows only contain *1* trial type. 
            # If so, this means we have max_iter rows of the same trials, and we need to change something.
            if data.iloc[row_selection][column].nunique() == 1:
                return True

        return False

    def run(self, debug=False):
        """
        Pseudorandomizes: makes sure that it is not possible to have more than x iterations for every type of column, specified in columns.
        """
        
        print("pseudorandomizer running")
        
        # Start by copying from original data, and shuffle
        self.data = self.data.sample(frac=1, random_state=np.random.randint(0, 1e7, dtype='int')).reset_index(drop=True) 
        
        if debug:
            outer_while_i = 0
            debug_print_after_i = 100

        good_set = False
        while not good_set:
            if debug:
                outer_while_i += 1

            reshuffle = False  # Assume the dataset does not need reshuffling.
            for row_n in range(0, self.data.shape[0]):

                # Select rows [max_iter-1 - row_n] we're going to check

                # Check if the current row, and the (max_iters-1) rows before, are the same value (number of unique values = 1).
                # If so, then move the current row number to the bottom of the dataframe. However, we need to re-check the same four rows again
                # after moving a row to the bottom: therefore, a while loop is necessary.
                checked_row = False
                n_attempts_at_moving = 0
                
                if debug:
                    inner_while_i = 0
                
                while not checked_row:
                    if debug:
                        inner_while_i += 1
                        if inner_while_i > debug_print_after_i:
                            print('New inner loop started for current row')

                    if self.check_trial_rows(self.data, row_n):
                        if debug and inner_while_i > debug_print_after_i:
                            print('Found too many consecutively identical rows.')

                        # If there are too many consecutively identical rows at the bottom of the dataframe, 
                        # break and start over/shuffle
                        if row_n >= (self.data.shape[0] - self.max_identical_iters[list(self.max_identical_iters.keys())[0]]):
                            if debug and inner_while_i > debug_print_after_i:
                                print(('These occurred at row_n %d, which is at the bottom of the DF.' % row_n))

                            checked_row = True
                            reshuffle = True

                        # Too many consecutive identical rows? Move row_n to the bottom, and check again with the new row_n.
                        else:
                            if debug and inner_while_i > debug_print_after_i:
                                print(('These occurred at row_n %d. Checking the remainder of the DF.' % row_n))

                            # Check if moving to the bottom even makes sense: if all remaining values are identical, it doesn't.
                            if (self.data.iloc[row_n:][list(self.max_identical_iters.keys())].nunique().values < 2).any():
                                if debug and inner_while_i > debug_print_after_i:
                                    print('All remaining values are identical. I should stop the for-loop, and start over.')

                                checked_row = True
                                reshuffle = True
                            else:
                                if n_attempts_at_moving < 50:
                                    n_attempts_at_moving += 1

                                    if debug and inner_while_i > debug_print_after_i:
                                        print('Not all remaining values are identical. I should move the final part to the bottom.')

                                    # If not, move the current row to the bottom
                                    row_to_move = self.data.iloc[row_n,:]

                                    # Delete row from df
                                    self.data.drop(row_n, axis=0, inplace=True)

                                    # Append original row to end. Make sure to reset index
                                    self.data = self.data.append(row_to_move).reset_index(drop=True)

                                # If we already tried moving the current row to the bottom for 50 times, let's forget about it and restart
                                else:
                                    checked_row = True
                                    reshuffle = True
                    else:
                        if debug and inner_while_i > debug_print_after_i:
                            print('Checked row, but the row is fine. Next row.')
                        checked_row = True

                if reshuffle:
                    good_set = False
                    break  # out of the for loop

                # Reached the bottom of the dataframe, but no reshuffle call? Then we're set.
                if row_n == self.data.shape[0]-1:
                    good_set = True

            if reshuffle:
                # Shuffle, reset index to ensure trial_data.drop(row_n) rows
                self.data = self.data.sample(frac=1, random_state=np.random.randint(0, 1e7, dtype='int')).reset_index(drop=True)
        
        print("Finished Pseudorandomizer")
        return self.data

# add in null trials class
def add_pseudorandom_null_trials(data, min_row=4, max_row=4, min_n_rows_separate=7, 
                                n_null_trials=10, null_column_name=''):
    """ 
    Adds null trials interspersed at pseudorandom locations. You can determine the minimum
    number of trials at the start before a null trial, the minimum number of trials at the end in which no
    nulls are shown, and the minimum number of trials that the null trials have to be separated 
    """
    
    print("Adding null trials")
    
    if n_null_trials != 0:
        n_null_trials = n_null_trials-1
    
    good_idx = False
    while not good_idx:
        indx = np.random.choice(np.arange(min_row, data.shape[0]-max_row), 
                                replace=False, size=n_null_trials)
        diffs = np.diff(np.sort(indx))
        if (diffs >= min_n_rows_separate).all():
            good_idx = True
    
    data.index = np.setdiff1d(np.arange(data.shape[0] + n_null_trials), indx)
    new_rows = pd.DataFrame({null_column_name: [True]*n_null_trials}, columns=data.columns, index=indx)    
    data = data.append(new_rows).sort_index()
    
    # Always end with a null trial
    if n_null_trials != 0:
        last_row = pd.DataFrame({null_column_name: [True]*1}, columns=data.columns, index=[data.shape[0]])
        data = data.append(last_row).sort_index() 
    
    print("Finished adding null trials")
    return data

# Generate main designs

Set parameters for the main design generation

In [36]:
# set paramters for main design generation
# For standard experiment main parameters should be: p_null_trials=0.1, p_stop_trial=0.25

n_subjects = 72                 # How many subjects to make a design for
tr = 1.4                        # What TR will be used in the fMRI experiment
n_blocks = 2                    # How many blocks per session 
n_sessions = 1                  # How many sessions will there be
n_trials_per_block=111          # How many trials are there in each block
p_null_trials=0.1               # Set the amount of null trials you want
p_stop_trial = 0.25             # Set the probabilty of stop trials
volumes_per_trial = 5           # How many TRs should there be per trial
prac_trials = 30                # How many practice trials do you want
prac_stop_trials = prac_trials*p_stop_trial   # How many practice stop trials should there be
trial_length = tr*volumes_per_trial           # The length of the trial is dependent on TR and number of volumes that you want
stimulus_duration = 1.2                         # how long should the stimulus be shown

Generate the design for the main experiment with practice trials

In [41]:
# Define the settings for the designs    
settings = get_settings(tr=tr, n_blocks=n_blocks, n_sessions=n_sessions, n_trials_per_block=n_trials_per_block, p_null_trials=p_null_trials, 
                        p_stop_trial=p_stop_trial, volumes_per_trial=volumes_per_trial, verbose=True)
settings

Settings:

        Sessions: 1
        Blocks per session: 2
        Trials per block: 111
        Total trials entire session: 222
        Stop trials per block: 25 (25.00 percent)
        Number of null trials per block: 0.1*111 = 11
        Assuming a TR of 1.4 seconds
        Jitter options: [0.5, 1.0, 1.5, 2.0] seconds
        Total duration: 1554.0 seconds
        Total duration: 25.9 minutes
        Total volumes: 1111 volumes
        Total stop trials: 25*2 = 50
        Total duration of one block: 7.0*111 = 1554.0 seconds = 12.95 min
        Total number of volumes necessary for 1 block: 1+111*5 = 1111 + warm-up pulses


{'jitter': [0.5, 1.0, 1.5, 2.0],
 'n_stop_trials': 25,
 'n_go_trials': 75,
 'n_trials_per_block': 111,
 'n_trials_per_block_not_null': 100,
 'n_null_trials': 11,
 'n_blocks': 2,
 'session_tr': 1.4,
 'trial_length': 7.0,
 'n_trs': 5}

In [43]:
# create the designs
design_folder = 'designs_SST'
save_dir = opj(os.getcwd(), design_folder)

print('You are about to save these designs in: ' + save_dir) 
print('Check that this is correct')

# Ask if user wants to continue
ans=[]
while ans not in ['y','yes','Yes','n','no','No']:
    ans = input('Continue? (y or n): ')

# stop code if in wrong directory
if ans in ['n','no','No']:
    os.chdir(input('input path here: '))
    save_dir = os.getcwd()

# set save directory
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
    
# loop through and create designs for all subjects, with practice design
for subject_id in np.arange(1,n_subjects+1):
    # Loop over sessions
    for n_sess in range(1,n_sessions+1):
        # Loop over blocks
        designs_this_session = []

        for run in range(1,n_blocks+1):

            design = generate_design(settings['n_trials_per_block'], settings['jitter'], settings['n_stop_trials'], settings['n_null_trials'])

            print("generating block " + str(run) + ' for subject ' + str(subject_id))

            # Pseduoranomize order and add null trials
            updated_design = add_pseudorandom_null_trials(Pseudorandomizer(design, max_identical_iters={'direction': 4, 'jitter': 3}).run(), 
                                                          min_row=round(settings['n_null_trials']*0.6), max_row=round(settings['n_null_trials']*0.6),
                                                          min_n_rows_separate=round(settings['n_null_trials']*0.6), 
                                                          n_null_trials=settings['n_null_trials'], null_column_name='null_trial')

            updated_design['block'] = run
            designs_this_session.append(updated_design)

            # add bllock number
            design['block'] = run

            # output that the block is finished
            print('completed block\n')

        ####### main design ######
        design = pd.concat(designs_this_session).reset_index(drop=True)
        design['subject'] = subject_id
        design['stimulus_dur'] = stimulus_duration
        design['feedback_dur'] = 0.0

        # add randomized jitter times to null trials        
        design.loc[design.isnull().any(axis=1),'jitter'] = np.random.choice(settings['jitter'], 
                                                                            size=settings['n_null_trials']*settings['n_blocks'], 
                                                                            replace=True)
        
        design['null_trial'] = 0
        design.loc[design.isnull().any(axis=1),'null_trial'] = 1

        #design.loc[design.isnull().any(axis=1),'jitter'] = 0.75 # set all null trials jitter to 0.75s
        design['ITI'] = (trial_length - stimulus_duration - design['jitter']).round(3)
        design['n_trs'] = settings['n_trs']

        fn = f'sub-{str(subject_id).zfill(3)}_ses-{str(n_sess)}_tr-{str(tr)}_design_task-SST'
        print(fn)
        design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')

        ###### practice design ######
        design_prac = generate_design(prac_trials,settings['jitter'],prac_stop_trials, n_null_trials=None)
        design_prac['block'] = 1
        design_prac['subject'] = subject_id
        design_prac['stimulus_dur'] = stimulus_duration
        design_prac['feedback_dur'] = 0.5
        design_prac['null_trial'] = 0
        design_prac['ITI'] = (trial_length - stimulus_duration - design_prac['jitter'] - design_prac['feedback_dur']).round(3)
        design_prac['n_trs'] = settings['n_trs']

        fn_prac = f'sub-p{str(subject_id).zfill(3)}_ses-{str(n_sess)}_tr-{str(tr)}_design_task-SST'
        print(fn_prac)
        design_prac.to_csv(save_dir + '/' + fn_prac + '.csv', sep='\t', index_label='trial_ID')

You are about to save these designs in: /Users/scotti/surfdrive/Shared/NTNU_fMRI_tasks/tasks_for_at_NTNU/IMCN-SST-visual/designs_SST
Check that this is correct
Continue? (y or n): y
generating block 1 for subject 1
pseudorandomizer running
Finished Pseudorandomizer
Adding null trials
Finished adding null trials
completed block

generating block 2 for subject 1
pseudorandomizer running
Finished Pseudorandomizer
Adding null trials
Finished adding null trials
completed block

22
11
sub-001_ses-1_tr-1.4_design_task-SST
sub-p001_ses-1_tr-1.4_design_task-SST
generating block 1 for subject 2
pseudorandomizer running
Finished Pseudorandomizer
Adding null trials
Finished adding null trials
completed block

generating block 2 for subject 2
pseudorandomizer running
Finished Pseudorandomizer
Adding null trials
Finished adding null trials
completed block

22
11
sub-002_ses-1_tr-1.4_design_task-SST
sub-p002_ses-1_tr-1.4_design_task-SST
generating block 1 for subject 3
pseudorandomizer running
Finish

Finished adding null trials
completed block

generating block 2 for subject 22
pseudorandomizer running
Finished Pseudorandomizer
Adding null trials
Finished adding null trials
completed block

22
11
sub-022_ses-1_tr-1.4_design_task-SST
sub-p022_ses-1_tr-1.4_design_task-SST
generating block 1 for subject 23
pseudorandomizer running
Finished Pseudorandomizer
Adding null trials
Finished adding null trials
completed block

generating block 2 for subject 23
pseudorandomizer running
Finished Pseudorandomizer
Adding null trials
Finished adding null trials
completed block

22
11
sub-023_ses-1_tr-1.4_design_task-SST
sub-p023_ses-1_tr-1.4_design_task-SST
generating block 1 for subject 24
pseudorandomizer running
Finished Pseudorandomizer
Adding null trials
Finished adding null trials
completed block

generating block 2 for subject 24
pseudorandomizer running
Finished Pseudorandomizer
Adding null trials
Finished adding null trials
completed block

22
11
sub-024_ses-1_tr-1.4_design_task-SST
sub-p

Finished adding null trials
completed block

generating block 2 for subject 44
pseudorandomizer running
Finished Pseudorandomizer
Adding null trials
Finished adding null trials
completed block

22
11
sub-044_ses-1_tr-1.4_design_task-SST
sub-p044_ses-1_tr-1.4_design_task-SST
generating block 1 for subject 45
pseudorandomizer running
Finished Pseudorandomizer
Adding null trials
Finished adding null trials
completed block

generating block 2 for subject 45
pseudorandomizer running
Finished Pseudorandomizer
Adding null trials
Finished adding null trials
completed block

22
11
sub-045_ses-1_tr-1.4_design_task-SST
sub-p045_ses-1_tr-1.4_design_task-SST
generating block 1 for subject 46
pseudorandomizer running
Finished Pseudorandomizer
Adding null trials
Finished adding null trials
completed block

generating block 2 for subject 46
pseudorandomizer running
Finished Pseudorandomizer
Adding null trials
Finished adding null trials
completed block

22
11
sub-046_ses-1_tr-1.4_design_task-SST
sub-p

Finished Pseudorandomizer
Adding null trials
Finished adding null trials
completed block

generating block 2 for subject 66
pseudorandomizer running
Finished Pseudorandomizer
Adding null trials
Finished adding null trials
completed block

22
11
sub-066_ses-1_tr-1.4_design_task-SST
sub-p066_ses-1_tr-1.4_design_task-SST
generating block 1 for subject 67
pseudorandomizer running
Finished Pseudorandomizer
Adding null trials
Finished adding null trials
completed block

generating block 2 for subject 67
pseudorandomizer running
Finished Pseudorandomizer
Adding null trials
Finished adding null trials
completed block

22
11
sub-067_ses-1_tr-1.4_design_task-SST
sub-p067_ses-1_tr-1.4_design_task-SST
generating block 1 for subject 68
pseudorandomizer running
Finished Pseudorandomizer
Adding null trials
Finished adding null trials
completed block

generating block 2 for subject 68
pseudorandomizer running
Finished Pseudorandomizer
Adding null trials
Finished adding null trials
completed block

22


# generate 1010 design just for debugging

Set parameters for debugging design

In [24]:
# For standard experiment main parameters should be: p_null_trials=0.1, p_stop_trial=0.25

subject_id = 1010
tr = 1.4                        # What TR will be used in the fMRI experiment
n_blocks = 3                    # How many blocks per session 
n_sessions = 1                  # How many sessions will there be
n_trials_per_block=6          # How many trials are there in each block
p_null_trials=0               # Set the amount of null trials you want
p_stop_trial = 0.25             # Set the probabilty of stop trials
volumes_per_trial = 6           # How many TRs should there be per trial
prac_trials = 6                # How many practice trials do you want
prac_stop_trials = prac_trials*p_stop_trial   # How many practice stop trials should there be
trial_length = tr*volumes_per_trial           # The length of the trial is dependent on TR and number of volumes that you want
stimulus_duration = 1.2                         # how long should the stimulus be shown

In [25]:
settings = get_settings(tr=tr, n_blocks=n_blocks, n_sessions=n_sessions, n_trials_per_block=n_trials_per_block, p_null_trials=p_null_trials, 
                        p_stop_trial=p_stop_trial, volumes_per_trial=volumes_per_trial, verbose=True)
settings

Settings:

        Sessions: 1
        Blocks per session: 3
        Trials per block: 6
        Total trials entire session: 18
        Stop trials per block: 1 (16.67 percent)
        Number of null trials per block: 0*6 = 0
        Assuming a TR of 1.4 seconds
        Jitter options: [0.5, 1.0, 1.5, 2.0] seconds
        Total duration: 151.2 seconds
        Total duration: 2.52 minutes
        Total volumes: 109 volumes
        Total stop trials: 1*3 = 3
        Total duration of one block: 8.4*6 = 151.2 seconds = 0.84 min
        Total number of volumes necessary for 1 block: 1+6*6 = 109 + warm-up pulses


{'jitter': [0.5, 1.0, 1.5, 2.0],
 'n_stop_trials': 1,
 'n_go_trials': 5,
 'n_trials_per_block': 6,
 'n_trials_per_block_not_null': 6,
 'n_null_trials': 0,
 'n_blocks': 3,
 'session_tr': 1.4,
 'trial_length': 8.4,
 'n_trs': 6}

In [26]:
# create the designs
design_folder = 'designs_SST'
save_dir = opj(os.getcwd(), design_folder)

print('You are about to save these designs in: ' + save_dir) 
print('Check that this is correct')

# Ask if user wants to continue
ans=[]
while ans not in ['y','yes','Yes','n','no','No']:
    ans = input('Continue? (y or n): ')

# stop code if in wrong directory
if ans in ['n','no','No']:
    os.chdir(input('input path here: '))
    save_dir = os.getcwd()

# set save directory
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

# Create design
designs_this_session = []

# Loop over sessions
for n_sess in range(1,n_sessions+1):
    # Loop over blocks
    for run in range(1,n_blocks+1):

        design = generate_design(settings['n_trials_per_block'], settings['jitter'], settings['n_stop_trials'], settings['n_null_trials'])
        design['block'] = run

        designs_this_session.append(design) 

    # concatenate designs
    design = pd.concat(designs_this_session).reset_index(drop=True)
    design['subject'] = subject_id
    design['stimulus_dur'] = stimulus_duration
    design['feedback_dur'] = 0
    design['null_trial'] = 0
    design['ITI'] = (trial_length - stimulus_duration - design['jitter']).round(3)
    design['n_trs'] = settings['n_trs']

    # Save deisgn
    fn = f'sub-{str(subject_id).zfill(3)}_ses-{str(n_sess)}_tr-{str(tr)}_design_task-SST'
    print(fn)
    design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')

    # Creat prac design
    design_prac = generate_design(prac_trials,settings['jitter'],prac_stop_trials, n_null_trials=None)
    design_prac['block'] = 1
    design_prac['subject'] = subject_id
    design_prac['stimulus_dur'] = stimulus_duration
    design_prac['feedback_dur'] = 0.5
    design_prac['null_trial'] = 0
    design_prac['ITI'] = (trial_length - stimulus_duration - design_prac['jitter'] - design_prac['feedback_dur']).round(3)
    design_prac['n_trs'] = settings['n_trs']
    
    # save prac design
    fn_prac = f'sub-p{str(subject_id).zfill(3)}_ses-{str(n_sess)}_tr-{str(tr)}_design_task-SST'
    print(fn_prac)
    design_prac.to_csv(save_dir + '/' + fn_prac + '.csv', sep='\t', index_label='trial_ID')

You are about to save these designs in: /Users/scotti/surfdrive/Projects/7T_SST_MSIT_NTNU/IMCN-SST-visual/designs_SST
Check that this is correct
Continue? (y or n): y
sub-1010_ses-1_tr-1.4_design_task-SST
sub-p1010_ses-1_tr-1.4_design_task-SST


# Generate designs for pilot MRI session

In [37]:
# set paramters for main design generation
# For standard experiment main parameters should be: p_null_trials=0.1, p_stop_trial=0.25

subject_id = 999                     # What ID do you want for the pilot
tr = 1.4                        # What TR will be used in the fMRI experiment
n_blocks = 1                    # How many blocks per session 
n_sessions = 1                  # How many sessions will there be
n_trials_per_block = 36          # How many trials are there in each block
p_null_trials = 0.0               # Set the amount of null trials you want
p_stop_trial = 0.25             # Set the probabilty of stop trials (e.g., 0.25 or 0.3, 0, or 1)
volumes_per_trial = 6           # How many TRs should there be per trial
prac_trials = 30                # How many practice trials do you want
prac_stop_trials = prac_trials*p_stop_trial   # How many practice stop trials should there be
trial_length = tr*volumes_per_trial           # The length of the trial is dependent on TR and number of volumes that you want
stimulus_duration = 1.2                         # how long should the stimulus be shown

In [38]:
# define the TR you want to use and check the settings
settings = get_settings(tr=tr, n_blocks=n_blocks, n_sessions=n_sessions, n_trials_per_block=n_trials_per_block, p_null_trials=p_null_trials, 
                        p_stop_trial=p_stop_trial, volumes_per_trial=volumes_per_trial, verbose=True)
settings

Settings:

        Sessions: 1
        Blocks per session: 1
        Trials per block: 36
        Total trials entire session: 36
        Stop trials per block: 9 (25.00 percent)
        Number of null trials per block: 0.0*36 = 0
        Assuming a TR of 1.4 seconds
        Jitter options: [0.5, 1.0, 1.5, 2.0] seconds
        Total duration: 302.4 seconds
        Total duration: 5.04 minutes
        Total volumes: 217 volumes
        Total stop trials: 9*1 = 9
        Total duration of one block: 8.4*36 = 302.4 seconds = 5.04 min
        Total number of volumes necessary for 1 block: 1+36*6 = 217 + warm-up pulses


{'jitter': [0.5, 1.0, 1.5, 2.0],
 'n_stop_trials': 9,
 'n_go_trials': 27,
 'n_trials_per_block': 36,
 'n_trials_per_block_not_null': 36,
 'n_null_trials': 0,
 'n_blocks': 1,
 'session_tr': 1.4,
 'trial_length': 8.4,
 'n_trs': 6}

In [39]:
# create the designs
design_folder = 'designs_SST'
save_dir = opj(os.getcwd(), design_folder)

print('You are about to save these designs in: ' + save_dir) 
print('Check that this is correct')

# Ask if user wants to continue
ans=[]
while ans not in ['y','yes','Yes','n','no','No']:
    ans = input('Continue? (y or n): ')

# stop code if in wrong directory
if ans in ['n','no','No']:
    os.chdir(input('input path here: '))
    save_dir = os.getcwd()

# set save directory
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

designs_this_session = []

# Loop over sessions
for n_sess in range(1,n_sessions+1):
    # Loop over blocks
    for run in range(1,n_blocks+1):

        design_pilot = generate_design(settings["n_trials_per_block"], settings["jitter"], settings["n_stop_trials"], settings["n_null_trials"])

        print("generating block " + str(run) + ' for subject ' + str(subject_id))

        # Pseduoranomize order and add null trials
        updated_design = add_pseudorandom_null_trials(Pseudorandomizer(design_pilot, max_identical_iters={'direction': 4, 'jitter': 3}).run(), 
                                                      min_row=round(settings['n_null_trials']*0.6), max_row=round(settings['n_null_trials']*0.6), 
                                                      min_n_rows_separate=round(settings['n_null_trials']*0.6), 
                                                      n_null_trials=settings['n_null_trials'], null_column_name='null_trial')

        updated_design['block'] = run

        designs_this_session.append(updated_design)

    ####### main design ######
    design = pd.concat(designs_this_session).reset_index(drop=True)
    design['subject'] = subject_id
    design['stimulus_dur'] = stimulus_duration
    design['feedback_dur'] = 0

    jitts_to_add = np.random.choice(settings["jitter"],size=settings['n_null_trials']*n_blocks,replace=True)
    design.loc[design.isnull().any(axis=1),'jitter'] = jitts_to_add
    design['null_trial'] = 0
    design.loc[design.isnull().any(axis=1),'null_trial'] = 1

    #design.loc[design.isnull().any(axis=1),'jitter'] = 0.75 # set all null trials jitter to 0.75s
    design['ITI'] = (trial_length - stimulus_duration - design['jitter']).round(3)
    design['n_trs'] = volumes_per_trial

    fn = f'sub-{str(subject_id).zfill(3)}_ses-{str(n_sess)}_tr-{str(tr)}_design_task-SST'
    print(fn)
    design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')

    # Creat prac design
    design_prac = generate_design(prac_trials,settings['jitter'],prac_stop_trials, n_null_trials=None)
    design_prac['block'] = 1
    design_prac['subject'] = subject_id
    design_prac['stimulus_dur'] = stimulus_duration
    design_prac['feedback_dur'] = 0.5
    design_prac['null_trial'] = 0
    design_prac['ITI'] = (trial_length - stimulus_duration - design_prac['jitter']).round(3)
    design_prac['n_trs'] = volumes_per_trial

    # save prac design
    fn_prac = f'sub-p{str(subject_id).zfill(3)}_ses-{str(n_sess)}_tr-{str(tr)}_design_task-SST'
    print(fn_prac)
    design_prac.to_csv(save_dir + '/' + fn_prac + '.csv', sep='\t', index_label='trial_ID')

You are about to save these designs in: /Users/scotti/surfdrive/Projects/7T_SST_MSIT_NTNU/IMCN-SST-visual/designs_SST
Check that this is correct
Continue? (y or n): y
generating block 1 for subject 999
pseudorandomizer running
Finished Pseudorandomizer
Adding null trials
Finished adding null trials
sub-999_ses-1_tr-1.4_design_task-SST
sub-p999_ses-1_tr-1.4_design_task-SST


# Create quick behavioural only design for quick practicing

In [34]:
# set paramters for main design generation
# For standard experiment main parameters should be: p_null_trials=0.1, p_stop_trial=0.25

subject_id = 111                     # What ID do you want for the pilot
tr = 1.4                        # What TR will be used in the fMRI experiment
n_blocks = 1                    # How many blocks per session 
n_sessions = 1                  # How many sessions will there be
n_trials_per_block = 80          # How many trials are there in each block
p_null_trials = 0.0               # Set the amount of null trials you want
p_stop_trial = 0.25             # Set the probabilty of stop trials (e.g., 0.25 or 0.3, 0, or 1)
volumes_per_trial = 3           # How many TRs should there be per trial
prac_trials = 30                # How many practice trials do you want
prac_stop_trials = prac_trials*p_stop_trial   # How many practice stop trials should there be
trial_length = tr*volumes_per_trial           # The length of the trial is dependent on TR and number of volumes that you want
stimulus_duration = 1.2                         # how long should the stimulus be shown

In [35]:
# define the TR you want to use and check the settings
settings = get_settings(tr=tr, n_blocks=n_blocks, n_sessions=n_sessions, n_trials_per_block=n_trials_per_block, p_null_trials=p_null_trials, 
                        p_stop_trial=p_stop_trial, volumes_per_trial=volumes_per_trial, verbose=True)
settings

Settings:

        Sessions: 1
        Blocks per session: 1
        Trials per block: 80
        Total trials entire session: 80
        Stop trials per block: 20 (25.00 percent)
        Number of null trials per block: 0.0*80 = 0
        Assuming a TR of 1.4 seconds
        Jitter options: [0.5, 1.0, 1.5, 2.0] seconds
        Total duration: 336.0 seconds
        Total duration: 5.6 minutes
        Total volumes: 241 volumes
        Total stop trials: 20*1 = 20
        Total duration of one block: 4.2*80 = 336.0 seconds = 5.6 min
        Total number of volumes necessary for 1 block: 1+80*3 = 241 + warm-up pulses


{'jitter': [0.5, 1.0, 1.5, 2.0],
 'n_stop_trials': 20,
 'n_go_trials': 60,
 'n_trials_per_block': 80,
 'n_trials_per_block_not_null': 80,
 'n_null_trials': 0,
 'n_blocks': 1,
 'session_tr': 1.4,
 'trial_length': 4.2,
 'n_trs': 3}

In [36]:
# create the designs
design_folder = 'designs_SST'
save_dir = opj(os.getcwd(), design_folder)

print('You are about to save these designs in: ' + save_dir) 
print('Check that this is correct')

# Ask if user wants to continue
ans=[]
while ans not in ['y','yes','Yes','n','no','No']:
    ans = input('Continue? (y or n): ')

# stop code if in wrong directory
if ans in ['n','no','No']:
    os.chdir(input('input path here: '))
    save_dir = os.getcwd()

# set save directory
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

designs_this_session = []

# Loop over sessions
for n_sess in range(1,n_sessions+1):
    # Loop over blocks
    for run in range(1,n_blocks+1):

        design_pilot = generate_design(settings["n_trials_per_block"], settings["jitter"], settings["n_stop_trials"], settings["n_null_trials"])

        print("generating block " + str(run) + ' for subject ' + str(subject_id))

        # Pseduoranomize order and add null trials
        updated_design = add_pseudorandom_null_trials(Pseudorandomizer(design_pilot, max_identical_iters={'direction': 4, 'jitter': 3}).run(), 
                                                      min_row=round(settings['n_null_trials']*0.63), max_row=round(settings['n_null_trials']*0.63), 
                                                      min_n_rows_separate=round(settings['n_null_trials']*0.63), 
                                                      n_null_trials=settings['n_null_trials'], null_column_name='null_trial')

        updated_design['block'] = run

        designs_this_session.append(updated_design)

    ####### main design ######
    design = pd.concat(designs_this_session).reset_index(drop=True)
    design['subject'] = subject_id
    design['stimulus_dur'] = stimulus_duration
    design['feedback_dur'] = 0

    jitts_to_add = np.random.choice(settings["jitter"],size=settings['n_null_trials']*n_blocks,replace=True)
    design.loc[design.isnull().any(axis=1),'jitter'] = jitts_to_add
    design['null_trial'] = 0
    design.loc[design.isnull().any(axis=1),'null_trial'] = 1

    #design.loc[design.isnull().any(axis=1),'jitter'] = 0.75 # set all null trials jitter to 0.75s
    design['ITI'] = (trial_length - stimulus_duration - design['jitter']).round(3)
    design['n_trs'] = volumes_per_trial

    fn = f'sub-{str(subject_id).zfill(3)}_ses-{str(n_sess)}_tr-{str(tr)}_design_task-SST'
    print(fn)
    design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')

    # Creat prac design
    design_prac = generate_design(prac_trials,settings['jitter'],prac_stop_trials, n_null_trials=None)
    design_prac['block'] = 1
    design_prac['subject'] = subject_id
    design_prac['stimulus_dur'] = stimulus_duration
    design_prac['feedback_dur'] = 0.5
    design_prac['null_trial'] = 0
    design_prac['ITI'] = (trial_length - stimulus_duration - design_prac['jitter']).round(3)
    design_prac['n_trs'] = volumes_per_trial

    # save prac design
    fn_prac = f'sub-p{str(subject_id).zfill(3)}_ses-{str(n_sess)}_tr-{str(tr)}_design_task-SST'
    print(fn_prac)
    design_prac.to_csv(save_dir + '/' + fn_prac + '.csv', sep='\t', index_label='trial_ID')

You are about to save these designs in: /Users/scotti/surfdrive/Projects/7T_SST_MSIT_NTNU/IMCN-SST-visual/designs_SST
Check that this is correct
Continue? (y or n): y
generating block 1 for subject 111
pseudorandomizer running
Finished Pseudorandomizer
Adding null trials
Finished adding null trials
sub-111_ses-1_tr-1.4_design_task-SST
sub-p111_ses-1_tr-1.4_design_task-SST


---------------------------- END OF NOTEBOOOK -------------------------------

-------------------------------- RANDOM CODE ------------------------------

In [51]:
st = str(1)
fns = 'sdmgds'
if st > 1 and len(fns) == 1:
    print(blah)

TypeError: '>' not supported between instances of 'str' and 'int'

In [75]:
block1 = design

In [77]:
block1

,direction,stop_trial,jitter,block,subject,stimulus_dur,feedback_dur,null_trial,ITI
0,1.0,0.0,2.0,1,222,2,0,0,1.0
1,1.0,0.0,1.0,1,222,2,0,0,2.0
2,0.0,0.0,1.5,1,222,2,0,0,1.5
3,0.0,0.0,0.5,1,222,2,0,0,2.5
4,1.0,0.0,1.0,1,222,2,0,0,2.0
5,0.0,1.0,1.5,1,222,2,0,0,1.5
6,0.0,1.0,0.5,1,222,2,0,0,2.5
7,1.0,0.0,2.0,1,222,2,0,0,1.0
8,NaN,NaN,0.5,1,222,2,0,1,2.5
9,1.0,0.0,2.0,1,222,2,0,0,1.0


In [79]:
block2 = design

In [80]:
block2

,direction,stop_trial,jitter,block,subject,stimulus_dur,feedback_dur,null_trial,ITI
0,1.0,1.0,1.0,1,222,2,0,0,2.0
1,0.0,1.0,0.5,1,222,2,0,0,2.5
2,0.0,0.0,1.5,1,222,2,0,0,1.5
3,0.0,1.0,1.5,1,222,2,0,0,1.5
4,0.0,0.0,1.5,1,222,2,0,0,1.5
5,1.0,0.0,2.0,1,222,2,0,0,1.0
6,1.0,0.0,1.0,1,222,2,0,0,2.0
7,0.0,0.0,0.5,1,222,2,0,0,2.5
8,1.0,0.0,2.0,1,222,2,0,0,1.0
9,0.0,0.0,0.5,1,222,2,0,0,2.5


In [81]:
block2['block'] = 2

In [84]:
new_design = pd.concat([block1,block2])

In [39]:
tester = new_design

In [83]:
b, c = block2.iloc[175].copy(), block2.iloc[191].copy()
block2.iloc[175], block2.iloc[191] = c,b

In [ ]:
a = pd.DataFrame(data = [[1,2],[3,4]], index=range(2), columns = ['A', 'B'])
b, c = a.iloc[0].copy(), a.iloc[1].copy()
a.iloc[0],a.iloc[1] = c,b

In [85]:
new_design['block'] = new_design['block'].astype(int)

In [86]:
new_design['subject'] = new_design['subject'].astype(int)
new_design['null_trial'] = new_design['null_trial'].astype(int)
new_design['feedback_dur'] = new_design['feedback_dur'].astype(int)
new_design['stimulus_dur'] = new_design['stimulus_dur'].astype(int)

In [88]:
new_design.to_csv('/Users/scotti/Downloads/SST_pilot_090620_reuse/IMCN-SST-MSIT/designs_SST/sub-222_tr-2.5_design.csv', sep='\t', index_label='trial_ID')


In [87]:
new_design

,direction,stop_trial,jitter,block,subject,stimulus_dur,feedback_dur,null_trial,ITI
0,1.0,0.0,2.0,1,222,2,0,0,1.0
1,1.0,0.0,1.0,1,222,2,0,0,2.0
2,0.0,0.0,1.5,1,222,2,0,0,1.5
3,0.0,0.0,0.5,1,222,2,0,0,2.5
4,1.0,0.0,1.0,1,222,2,0,0,2.0
5,0.0,1.0,1.5,1,222,2,0,0,1.5
6,0.0,1.0,0.5,1,222,2,0,0,2.5
7,1.0,0.0,2.0,1,222,2,0,0,1.0
8,NaN,NaN,0.5,1,222,2,0,1,2.5
9,1.0,0.0,2.0,1,222,2,0,0,1.0


In [ ]:
###### OLD ######

In [16]:
change = pd.read_csv('/Users/scotti/Downloads/SST_pilot_030720/IMCN-SST-pilot/designs_SST/sub-555_tr-1.6_design copy.csv',sep='\t')

In [20]:
change.to_csv('/Users/scotti/Downloads/SST_pilot_030720/IMCN-SST-pilot/designs_SST/sub-555_tr-1.6_design copy.csv', sep='\t', index_label='trial_ID')

In [18]:
change.trial_ID = range(0,384)

In [19]:
change.tail()

,trial_ID,direction,stop_trial,jitter,block,subject,stimulus_dur,feedback_dur,null_trial,ITI,n_trs
379,379,0.0,0.0,2.0,3,555,1,0,0,5.0,5
380,380,1.0,0.0,0.5,3,555,1,0,0,6.5,5
381,381,1.0,0.0,1.0,3,555,1,0,0,6.0,5
382,382,1.0,0.0,0.5,3,555,1,0,0,6.5,5
383,383,NaN,NaN,2.0,3,555,1,0,1,5.0,5


In [11]:
list(range(0,383))

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [ ]:
# # function to constrain occurances of null trials
# # def gen_nulls(n_trials_per_block, p_null_trials, design):
    
# #     num_nulls = round(n_trials_per_block*p_null_trials)
# #     too_close = True
# #     while too_close == True:
# #         which_trials = np.random.choice(range(0,n_trials_per_block-3), num_nulls, replace=False)   # generate null trial locations
# #         which_trials.sort()     # sort the generated null trial locations
# #         test_which = False      # set variable to false to start with
# #         len(design[design.]
        
# #         for i in range(1,round(n_trials_per_block*p_null_trials)):     # loop over number of required null trials
# #             if (which_trials[i] - which_trials[i-1]) <= 2:             # if the consecutive numbers are not different enough
# #                 test_which = True                                      # set variable to true
            
# #         if test_which == True:       # if numbers too close are found
# #             continue                 # find a new array of null trials
# #         elif test_which == False:    # if false
# #             too_close = False        # end the while loop
    
# #     print(which_trials)
# #     return which_trials

# # generate design function
# def generate_design(n_trials_per_block, jitters, n_stop_trials,n_null_trials):
#     # make direction array
    
#     # calculate number of required trials from input
# #     n_not_null_trials = round(n_trials_per_block*(1-p_null_trials))
# #     n_null_trials = round(n_trials_per_block*(p_null_trials))
# #     n_go_trials = round(0.75*n_not_null_trials)
# #     n_stop_trials = round(0.25*n_not_null_trials) 
    
#     n_not_null_trials = n_trials_per_block-n_null_trials
#     n_go_trials = n_trials_per_block-n_stop_trials-n_null_trials
    
    
#     print('Number of null trials: ' + str(n_null_trials))
#     print('Number of not null trials: ' + str(n_not_null_trials))
#     print('Number of go trials: ' + str(n_go_trials))
#     print('Number of stop trials: ' + str(n_stop_trials))
#     print('Number of total trials: ' + str(n_trials_per_block))
        
# #     while n_null_trials + n_go_trials + n_stop_trials > n_trials_per_block:
# #         n_go_trials -= 1
            
# #     while n_null_trials + n_go_trials + n_stop_trials < n_trials_per_block:
# #         n_stop_trials += 1
            
#     design = pd.DataFrame({'direction': np.tile([0, 1], int(np.ceil(n_not_null_trials/2))), # arrow direction
#                            'stop_trial': 0})#,                  # stop trial
#                            #'null_trial': 0})                 # null trial
#     design.loc[:(n_stop_trials-1), 'stop_trial'] = 1            # add correct number of stop trials
#     #design.loc[n_trials_per_block-n_null_trials-1:n_trials_per_block-1, 'null_trial'] = 1           # add correct number of null trials 
    
#     print(len(design))
#     print(str( int(n_not_null_trials/4)))
    
#     # make jitter array
#     design['jitter'] = np.tile(jitters, int(n_not_null_trials/4))
        
#     too_close = True
#     consec_stop = True
    
#     min_null = 4
#     max_null = 10
#     max_stop = 2
#     print('min trials inbetween null trials: ' + str(min_null))
#     print('max trials inbetween null trials: ' + str(max_null))    
#     print('min trials inbetween stop trials: ' + str(max_stop))
    
#     iterations = 1
    
#     # while loop to psuedorandomize null trial and stop trial separation
# #     while too_close==True or consec_stop==True:
        
# #         design = design.sample(frac=1).reset_index(drop=True) # Mix up design each while loop iteration
        
# #         test_which = False
# #         test_stop = False
        
# #         which_trials = np.where(design.null_trial) # which trials are null trials? 
# #         which_stop = np.where(design.stop_trial)   # which trials are stop trials? 
        
# #         # loop over null trials
# #         for i in range(1,len(which_trials[0])):
# #             print(((which_trials[0][i] - which_trials[0][i-1])))
# #             if ((which_trials[0][i] - which_trials[0][i-1]) <= min_null or (which_trials[0][i] - which_trials[0][i-1]) >= max_null):             # if the consecutive numbers are not different enough
# #                 test_which = True                                                   # then start the while loop again
                    
# #         #if test_which == True:       # if numbers too close are found
# #         #    continue                 # find a new array of null trials
# #         if test_which == False:       # if false
# #             too_close = False         # end the while loop
# #             print('end')
            
# #         # loop over stop trials
# #          for i in range(1,len(which_stop[0])):                            
# #              if (which_stop[0][i] - which_stop[0][i-1]) <= max_stop:             # if there are consecutive stop trials
# #                  test_stop = True                                                # then start the while loop again
                    
# #         #if test_stop == True:          # if numbers too close are found
# #         #    continue                   # find a new array of null trials
# #          if test_stop == False:          # if false
# #              consec_stop = False         # end the while loop
# #              print('end')
# #         consec_stop=False
            
# #         iterations += 1
            
#     print("number of iteration: " + str(iterations))
#     return(design)

### Generate design for practice
Use a single block of tr=2 settings

In [ ]:
save_dir = './designs'

# ORIGINAL PRACTICE DESIGN CODE IS HASHED BELOW
# settings = get_settings(tr=2)
# design = generate_design(settings['n_trials'], settings['jitter'], settings['n_stop_trials'])
# design['block'] = 1
# design = design.iloc[:20]  #only do 20 (changed from 75) trials; that takes 10min
# design
# fn = 'sub-practice_tr-' + str(2) + '_design'
# print(fn)
# design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')

for sub_id in np.arange(1,n_subjects+1):

        settings = get_settings(tr=2)
        design = generate_design(settings['n_trials'], settings['jitter'], settings['n_stop_trials'])
        design['block'] = 1
        design = design.iloc[:25]  #only do 20 (changed from 75) trials; that takes 10min
        
        design
        fn = 'sub-practice' + str(sub_id).zfill(3) + '_tr-' + str(2) + '_design'
        print(fn)
        design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')

In [ ]:
# create debug script

# setup DEBUG design
# 5 trials per block, 3 blocks (same amount as actual task)

#os.chdir('/Users/scotti/surfdrive/experiment_code/test_main_behavioural/merge_stopsig_MSIT_behavioural')
save_dir = './designs'

sub_id = 666 # this is debug 
session_tr=2
settings = get_settings(tr=2)

### normal debug design
# design = generate_design(settings['n_trials'], settings['jitter'], settings['n_stop_trials'])
                 
designs_this_session = []   
for run in range(1,n_runs+1):
    design = generate_design(settings['n_trials'], settings['jitter'], settings['n_stop_trials'])
    design['block'] = run
            
    designs_this_session.append(design)
        
design = pd.concat(designs_this_session)

block_nums = settings['n_trials']
design = design.iloc[pd.np.r_[0:5, block_nums:block_nums+5, 2*block_nums:2*block_nums+5]]
                     
fn = 'sub-' + str(sub_id).zfill(3) + '_tr-' + str(session_tr) + '_design'
print(fn)
design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID') 
                     
### Practice debug dfesign
design = generate_design(settings['n_trials'], settings['jitter'], settings['n_stop_trials'])
design['block'] = 1
design = design.iloc[:5]  #only do 20 (changed from 75) trials; that takes 10min
        
design
fn = 'sub-practice' + str(sub_id).zfill(3) + '_tr-' + str(2) + '_design'
print(fn)
design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')



### Generate design for pilot session
We do 4 runs here, with tr of 3, 2, 3, 2 (corresponding to ME, SE, ME, SE)

In [ ]:
designs = []
for block, tr in enumerate([3,2,3,2]):
    settings = get_settings(tr=tr, verbose=False)
    design = generate_design(settings['n_trials'], settings['jitter'], settings['n_stop_trials'])
    design['block'] = block+1
    designs.append(design)
design = pd.concat(designs)


fn = 'sub-pilot_tr-' + str(2) + '_design'
print(fn)
design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')

In [ ]:
design

### For debug?

In [ ]:
# for debug
n_runs = 3
n_sessions = 2
n_trials = 10
tr = 3
jitters = [0.75, 1.5, 2.25, 3]
n_stop_trials = int(np.ceil(n_trials/4))

save_dir = './designs'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
    
for subject_id in ['DEBUG']:
    for session_tr in range(2,4):
        designs_this_session = []
        
        for run in range(1,n_runs+1):
            settings = get_settings(session_tr)
            design = generate_design(n_trials, settings['jitter'][:n_trials], n_stop_trials)
            design['block'] = run
            design = design.iloc[:n_trials]
            
            designs_this_session.append(design)
        
        design = pd.concat(designs_this_session)
        fn = 'sub-' + str(subject_id).zfill(3) + '_tr-' + str(session_tr) + '_design'
        print(fn)
        design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')
design

In [ ]:
design.groupby('direction').stop_trial.sum()

# Finally, create designs for Presentation
This is only meant as a back-up. In case PsychoPy doesn't work out, we can run the experiment in Presentation. The code requires some input files defining, for every run, for every trial, the arrow direction and whether it's a stop trial

These files are tsv saved as txt, with two columns: direction (-1 / 1) and is_stop_trial (0 / 1)

In [ ]:
subject_id = 1
subject_id % 2

In [ ]:
tr_3_folder = './presentation_settings_tr3'
tr_2_folder = './presentation_settings_tr2'
os.makedirs(tr_3_folder)
os.makedirs(tr_2_folder)

for subject_id in range(1, n_subjects+1):
    if subject_id % 2 == 1:  # UNEVEN subjects start with ME, then SE
        settings = get_settings(tr=3)
        for run in range(1, 4):  # three runs
            design = generate_design(settings['n_trials'], settings['jitter'], settings['n_stop_trials'])
            design = design[['direction', 'stop_trial']]  # the other columns are not necessary but generated on the fly
            design.loc[design.direction == 0, 'direction'] = -1
            design.to_csv(tr_3_folder + '/stop_settings_S' + str(subject_id) + 's1r' + str(run) + '.txt', sep='\t', header=False, index=False)
        
        # Do the same but now tr=2
        settings = get_settings(tr=2)
        for run in range(1, 4):  # three runs
            design = generate_design(settings['n_trials'], settings['jitter'], settings['n_stop_trials'])
            design = design[['direction', 'stop_trial']]  # the other columns are not necessary but generated on the fly
            design.loc[design.direction == 0, 'direction'] = -1
            design.to_csv(tr_2_folder + '/stop_settings_S' + str(subject_id) + 's2r' + str(run) + '.txt', sep='\t', header=False, index=False)
    
    elif subject_id % 2 == 0:  # EVEN subjects: start with SE, then ME
        settings = get_settings(tr=2)
        for run in range(1, 4):  # three runs
            design = generate_design(settings['n_trials'], settings['jitter'], settings['n_stop_trials'])
            design = design[['direction', 'stop_trial']]  # the other columns are not necessary but generated on the fly
            design.loc[design.direction == 0, 'direction'] = -1
            design.to_csv(tr_2_folder + '/stop_settings_S' + str(subject_id) + 's1r' + str(run) + '.txt', sep='\t', header=False, index=False)

        settings = get_settings(tr=3)
        for run in range(1, 4):  # three runs
            design = generate_design(settings['n_trials'], settings['jitter'], settings['n_stop_trials'])
            design = design[['direction', 'stop_trial']]  # the other columns are not necessary but generated on the fly
            design.loc[design.direction == 0, 'direction'] = -1
            design.to_csv(tr_3_folder + '/stop_settings_S' + str(subject_id) + 's2r' + str(run) + '.txt', sep='\t', header=False, index=False)

In [ ]:
### Load everything to check
import glob
fns = glob.glob('./stop_settings_*.txt')

for fn in fns:
    sub_n = fn.split('_')[-1].split('.')[0]
    dat = pd.read_csv(fn, header=None, sep='\t')
    n_trials = dat.shape[0]
    n_stop_trials = dat.iloc[:,1].sum()
    print('File: %s, n trials: %d, n stop trials: %d' %(sub_n, n_trials, n_stop_trials))

In [ ]:
### Some tests
pilot_des = pd.read_csv('./designs/sub-pilot_tr-2_design.csv', sep='\t')

In [ ]:
pilot_des.groupby(['block']).jitter.unique()